#### Create necessary dependencies for reproducibility

In [ ]:
# Install a pip package in the current Jupyter kernel
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

# Get the current working directory
notebook_dir = os.getcwd()
# Set the notebook's folder as the working directory (if needed)
os.chdir(os.path.join(notebook_dir, ".."))

!{sys.executable} -m pip install -r config/requirements.txt

# Import required numpy libraries (default and custom)
import matplotlib.pyplot as plt
import numpy as np

from src.unsteady_2D import (
    compute_aerodynamics_static,
    compute_aerodynamics_unsteady,
    geometry_airfoil
)
import src.shared.utils

#### Define the inputs for the analysis

In [ ]:
# Flags
MOVEMENT_TYPES = ["flapping"]
# For MOVEMENT_TYPES, two options: "flapping" or "pitching"; can be combined in
# a single list
AERO_MODEL = "unsteady"  # two options: "steady", "quasi-steady" or "unsteady"
WAKE_MODEL = "free-wake"  # three options: "straight", "free-wake" or "prescribed"
STAGNATION_PHI_CHOICE = "LE"  # two options: "LE" or "TE"

# Define the inflow conditions
# Static angle of attack [rad] (only relevant if AERO_MODEL = "steady")
alpha_static = 0.0 * np.pi / 180
# Freestream velocity [m/s] in x-direction (approximately along streamwise)
U_inf = 0.13 * 340.2941
# Freestream velocity [m/s] in z-direction (approximately along normal)
W_inf = 0
rho = 1  # Density of working fluid [kg/m^3]

# Geometrical characteristics of the airfoil
c = 1  # Chord of the airfoil [m]
airfoil_type = "dat"  # Two options: "naca" or "dat"
# Name of the .dat airfoil where the airfoil coordinates are stored
airfoil_name = "DU95W180-LV.txt"
# Number of rows to skip in the airfoil file (if no skiprows, make it equal to -1)
airfoil_skiprows = -1
N = 200  # Number of panels (Only relevant if airfoil_type = "naca")

# Kinematics of the airfoil
# Characteristics of the periodic motion
N_cycles = 2.5  # Number of cycles
dt_per_cycle = 40  # Number of time steps per cycle

# Pitching variables
x_theta = 1.0 / 3.0 * c
z_theta = 0 * c
theta_min = 6.0 * np.pi / 180
theta_max = 6.0 * np.pi / 180
# Reduced frequencies (pitching) to be analyzed
k_theta_array = np.array([0])

# Flapping variables
# Hinge position
x_beta = 0.8 * c
z_beta = 0.0 * c
# Static flap angle sweep
beta_min = 0 * np.pi / 180
beta_max = 5 * np.pi / 180
# Number of flap sweep angles
n_beta_sweep = 10
# Reduced frequencies (flapping) to be analyzed
k_beta_array = np.array([0.05, 0.1, 0.5])

# Numerical settings of the model
# Coefficient telling how close the newly shed vortex should be to the TE
percentage = 0.25
# Small real number for the numerical calculation of the Jacobian
dx = 1e-8
rel_change_NVPM = 1e-10
max_N_iter_NVPM = 20

# Number of cycles used at the beginning of the simulation to reach a steady
# solution before starting the airfoil movement
N_cycles_startup = 1

PLOT_COLORS_LIST = ["k.", "b.", "g.", "c."]

#### Handle exceptions

In [ ]:
flapping_bool = "flapping" in MOVEMENT_TYPES
pitching_bool = "pitching" in MOVEMENT_TYPES

# Make sure that the chosen aerodynamic model is supported

if AERO_MODEL != "steady" and AERO_MODEL != "unsteady" and AERO_MODEL != "quasi-steady":
    raise ValueError(
        'Invalid AERO_MODEL, only possibilities are "unsteady" or "quasi-steady" '
    )

# Make sure that the chosen wake model is supported
if (
    WAKE_MODEL != "straight"
    and WAKE_MODEL != "free-wake"
    and WAKE_MODEL != "prescribed"
):
    raise ValueError(
        'Invalid WAKE_MODEL, only possibilities are "straight", "free-wake" or "prescribed" '
    )

# Set assumed values for the model if rotation variables are not relevant
if not pitching_bool:
    k_theta_array = np.zeros(k_beta_array.size)
    x_theta = 0.0
    theta_0 = 0.0
if not flapping_bool:
    x_beta = -1.0
    beta_0 = 0.0
    k_beta_array = np.zeros(k_theta_array.size)

# Ensure that the same number of reduced frequencies is provided for both pitching
# and flapping (necessary for the provided implementation)
if pitching_bool and flapping_bool:
    if np.size(k_theta_array) != np.size(k_beta_array):
        raise ValueError("k_theta_array and k_beta_array must have the same size")

if x_beta > 0 and x_theta > x_beta:
    raise ValueError(
        "For this code, the pitching axis must be closer to the LE"
        + " than the flapping one; make sure that x_theta < x_beta"
    )

if airfoil_type not in ["naca", "dat"]:
    raise ValueError(
        "Invalid value for airfoil_type; only 'naca' and 'dat' are supported"
    )

#### Organize static inputs into a single dictionary

To simplify the code syntax in the following, the relevant inputs are gathered within a dictionary that is used as input for the function computing the aerodynamic behaviour

In [ ]:
if airfoil_type == "dat":
    dat_additional_path = "data/raw/inputs/"
    airfoil_name = os.path.join(dat_additional_path, airfoil_name)
    (
        x_airfoil,
        z_airfoil,
        xc,
        zc,
        N,
        s,
        nx,
        nz,
        tx,
        tz,
        indexes_flap,
        hinge_position,
    ) = geometry_airfoil(
        airfoil_name,
        airfoil_type,
        np.array([x_beta, z_beta]),
        beta_input=np.mean([beta_min, beta_max]),
        desired_chord=c,
        x_pitch=x_theta,
        z_pitch=z_theta,
        skiprows=airfoil_skiprows,
    )
else:
    (
        x_airfoil,
        z_airfoil,
        xc,
        zc,
        N,
        s,
        nx,
        nz,
        tx,
        tz,
        indexes_flap,
        hinge_position,
    ) = geometry_airfoil(
        airfoil_name,
        airfoil_type,
        np.array([x_beta, z_beta]),
        beta_input=np.mean([beta_min, beta_max]),
        desired_chord=c,
        x_pitch=x_theta,
        z_pitch=z_theta,
        skiprows=airfoil_skiprows,
    )


steady_aerodynamics_inputs_dict = {
    "file_airfoil": airfoil_name,
    "type_airfoil": airfoil_type,
    "x_flap": x_beta,
    "z_flap": z_beta,
    "flap_angle": beta_min,
    "chord": c,
    "number_skiprows": airfoil_skiprows,
    "alpha_input": alpha_static,
    "freestream_velocity": U_inf,
}

#### Compute the static solution

In [ ]:
beta_vals = np.linspace(beta_min, beta_max, n_beta_sweep)
alpha_vals = np.ones(n_beta_sweep) * np.mean([theta_min, theta_max])
U_array = np.ones(n_beta_sweep) * U_inf
c_n_array = np.zeros(n_beta_sweep)
for i in range(n_beta_sweep):
    steady_aerodynamics_inputs_dict["flap_angle"] = beta_vals[i]
    steady_aerodynamics_inputs_dict["alpha_input"] = alpha_vals[i]
    steady_aerodynamics_inputs_dict["freestream_velocity"] = U_array[i]
    (C_p, c_n_array[i]) = compute_aerodynamics_static(steady_aerodynamics_inputs_dict)


#### Organize dynamic inputs into a single dictionary 

In [ ]:
# Amplitude / mean amplitude / phase (beta_0 / beta_mean / phi_beta) [rad]
beta_0 = (beta_max - beta_min) * 0.5
beta_mean = (beta_min + beta_max) * 0.5
phi_beta = 0 * np.pi / 180
flap_characteristics = np.array([beta_0, beta_mean, phi_beta, x_beta, z_beta])

theta_0 = (theta_max - theta_min) * 0.5
theta_mean = (theta_min + theta_max) * 0.5
phi_theta = 0.0 * np.pi / 180

# theta_0 = 0.0
# theta_mean = alpha_static
# phi_theta = 0.0

pitch_characteristics = np.array([theta_0, theta_mean, phi_theta, x_theta, z_theta])

total_N_cycles = N_cycles + N_cycles_startup
N_timesteps = int(total_N_cycles * dt_per_cycle + 1)
U_array = np.ones(int(total_N_cycles * dt_per_cycle + 1)) * U_inf
W_array = np.zeros(int(total_N_cycles * dt_per_cycle + 1))

unsteady_aerodynamics_inputs_dict = {
    "number_skiprows": airfoil_skiprows,
    "file_airfoil": airfoil_name,
    "type_airfoil": airfoil_type,
    "flap_or_not": flapping_bool,
    "pitch_or_not": pitching_bool,
    "flapping_parameters": flap_characteristics,
    "pitching_parameters": pitch_characteristics,
    "chord": c,
    "reduced_frequency_flap": k_beta_array[0],
    "reduced_frequency_pitch": k_theta_array[0],
    "U_inf": U_inf,
    "W_inf": W_inf,
    "timesteps_per_period": dt_per_cycle,
    "total_cycles": total_N_cycles,
    "startup_cycles": N_cycles_startup,
    "number_panels": N,
    "density": rho,
    "wake_type": WAKE_MODEL,
    "stagnation_potential_location": STAGNATION_PHI_CHOICE,
    "aerodynamic_model": AERO_MODEL,
    "iterations_Newton_TE_panel": max_N_iter_NVPM,
    "relative_change_crit_Newton_TE_panel": rel_change_NVPM,
    "dx_Newton_TE_panel": dx,
}

# Create list of np arrays to store the results
C_p_array_list = np.ndarray(shape=(len(k_beta_array), N, N_timesteps))
theta_store_list = np.ndarray(shape=(len(k_beta_array), N_timesteps))
beta_store_list = np.ndarray(shape=(len(k_beta_array), N_timesteps))
t_array_list = np.ndarray(shape=(len(k_beta_array), N_timesteps))
C_n_array_list = np.ndarray(shape=(len(k_beta_array), N_timesteps))

for q in range(len(k_beta_array)):
    unsteady_aerodynamics_inputs_dict["reduced_frequency_flap"] = k_beta_array[q]
    unsteady_aerodynamics_inputs_dict["reduced_frequency_pitch"] = k_theta_array[q]
    unsteady_aerodynamics_output_dict = compute_aerodynamics_unsteady(
        unsteady_aerodynamics_inputs_dict
    )
    t_array_list[q] = unsteady_aerodynamics_output_dict["time_instances"]
    theta_store_list[q] = unsteady_aerodynamics_output_dict["theta_values"]
    beta_store_list[q] = unsteady_aerodynamics_output_dict["beta_values"]
    C_p_array_list[q::] = unsteady_aerodynamics_output_dict["C_p"]
    C_n_array_list[q] = unsteady_aerodynamics_output_dict["C_n"]


#### Plot results, and compare to analytical solution

In [ ]:
# Compute the analytical solution for the lift coefficient
theta_x_beta = np.arccos(1 - 2 * x_beta / c)
Cl_values_thin_airfoil = (
    2 * (np.pi - theta_x_beta + np.sin(theta_x_beta)) * beta_vals
    + 2 * np.pi * alpha_vals
)
# remove .txt from airfoil_name if it has it
if ".txt" in airfoil_name:
    airfoil_name = airfoil_name[:-4]
# remove additional path from airfoil_name if it has it
if airfoil_type == "dat" and dat_additional_path in airfoil_name:
    airfoil_name = airfoil_name[16:]

plt.figure(1)
# plt.plot(beta_vals * 180 / np.pi, Cl_values_thin_airfoil, "r", label="Thin airfoil theory")
plt.plot(beta_vals * 180 / np.pi, c_n_array, "b", label="Steady Panel Method")
# plot each row of C_n_array, make it a dashed line
for i in range(len(k_beta_array)):
    if k_beta_array[i] != 0.3:
        indexes_plot = np.arange(
            beta_store_list[i].size - int((N_cycles - 0.5) * dt_per_cycle),
            beta_store_list[i].size,
        )
        plt.plot(
            beta_store_list[i][indexes_plot] * 180 / np.pi,
            C_n_array_list[i][indexes_plot],
            PLOT_COLORS_LIST[i],
            linestyle="--",
            label="k = " + str(k_beta_array[i]) + ", Panel Method, " + "DU95W180",
        )
plt.xlabel(r"$ \beta \ [^{\circ}]$")
plt.ylabel(r"$C_n$ [-]")
plt.legend()
plt.grid()
# Save high resolution plot in the folder "results/figures"
plt.savefig(
    "results/figures/panel_method_vs_thin_airfoil_theory_" + airfoil_name + ".png",
    dpi=300,
)

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Create the figure and axis
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 6))

theta_store = theta_store_list[0]
beta_store = beta_store_list[0]
C_p_array = C_p_array_list[0]
t_array = t_array_list[0]

# Plot setup: This line object will be updated in the animation
(line1,) = ax2.plot([], [], "ko", label="lower")  # First line
(line2,) = ax2.plot([], [], "bo", label="upper")  # Second line
(line3,) = ax1.plot([], [], "k-")  # Third line
(point,) = ax1.plot([], [], "ro")  # Plot setup for the point

if pitching_bool:
    angle_store = theta_store
    angle_mean = theta_mean
    angle_0 = theta_0
else:
    angle_store = beta_store
    angle_mean = beta_mean
    angle_0 = beta_0

# Configure graph with angle of attack

# Configure angle of attack graph
# Set the axis limits
ax1.set_xlim(0, 2 * t_array[-1] * U_inf / c)
ax1.set_ylim(
    np.rad2deg(angle_mean - 1.1 * angle_0), np.rad2deg(angle_mean + 1.1 * angle_0)
)

# Add legend and grid
ax1.grid()

ax1.set_xlabel(r"$\frac{t U_\infty}{c}$ [-]", fontsize=14)
if pitching_bool:
    ax1.set_ylabel(r"$\alpha \ [^{\circ}]$ ", fontsize=12)
else:
    ax1.set_ylabel(r"$\beta \ [^{\circ}]$ ", fontsize=12)

# Configure pressure distribution graph
# Set the axis limits
ax2.set_xlim(-0.1, 1.1)
ax2.set_ylim(-1.2, 5)

# Add legend and grid
ax2.legend()
ax2.grid()

ax2.set_xlabel(r"$\frac{x}{c}$ [-]", fontsize=14)
ax2.set_ylabel(r"$-C_p$ [-]", fontsize=12)

plt.subplots_adjust(hspace=0.4)  # Increase the vertical spacing


# Initialization function: to clear or set up the frame for the first time
def init():
    line1.set_data([], [])  # Clear the first line
    line2.set_data([], [])  # Clear the second line
    line3.set_data([], [])  # Clear the third line
    point.set_data([], [])  # Clear the point
    return line1, line2


# Animation function: called sequentially to update the plot
def update(frame):
    x1 = xc[0 : int(N / 2)] + x_theta
    y1 = -C_p_array[0 : int(N / 2), frame]
    line1.set_data(x1, y1)

    x2 = xc[int(N / 2) :] + x_theta
    y2 = -C_p_array[int(N / 2) :, frame]
    line2.set_data(x2, y2)

    line3.set_data(2 * t_array * U_inf / c, np.rad2deg(angle_store))

    point.set_data([2 * t_array[frame] * U_inf / c], [np.rad2deg(angle_store[frame])])

    return line1, line2


# Create the animation
ani = FuncAnimation(
    fig,  # The figure to animate
    update,  # The function that updates each frame
    frames=int(
        (N_cycles + N_cycles_startup) * dt_per_cycle
    ),  # Number of columns (frames)
    init_func=init,  # Initialization function
    blit=True,  # Use blitting for better performance
    interval=200,  # Delay between frames in milliseconds (adjust as needed)
)

HTML(ani.to_jshtml())
output_path = "results/figures"
# concatenate all components of movement types in a new variable
output_filename = MOVEMENT_TYPES[0]
if len(MOVEMENT_TYPES) > 1:
    output_filename += "_".join(MOVEMENT_TYPES[1:])
if pitching_bool:
    output_filename += "_k_theta_" + str(k_theta_array[0])
    output_filename += "_theta0_" + str(np.round(np.rad2deg(theta_0), 2))
    output_filename += "_theta_mean_" + str(np.round(np.rad2deg(theta_mean), 2))
if flapping_bool:
    output_filename += "_k_beta" + str(k_beta_array[0])
    output_filename += "_beta0_" + str(np.round(np.rad2deg(beta_0), 2))
    output_filename += "_beta_mean_" + str(np.round(np.rad2deg(beta_mean), 2))
output_path = os.path.join(output_path, output_filename + ".gif")
ani.save(output_path, writer="pillow", fps=5)
